<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 20s 761ms/step - loss: 0.6780 - accuracy: 0.5645 - val_loss: 0.6419 - val_accuracy: 0.7415
Epoch 2/10
26/26 [==============================] - 19s 727ms/step - loss: 0.6194 - accuracy: 0.6873 - val_loss: 0.5989 - val_accuracy: 0.6927
Epoch 3/10
26/26 [==============================] - 19s 731ms/step - loss: 0.5901 - accuracy: 0.6959 - val_loss: 0.6148 - val_accuracy: 0.5854
Epoch 4/10
26/26 [==============================] - 19s 723ms/step - loss: 0.5168 - accuracy: 0.7749 - val_loss: 0.4940 - val_accuracy: 0.7268
Epoch 5/10
26/26 [==============================] - 19s 719ms/step - loss: 0.4771 - accuracy: 0.7993 - val_loss: 0.3965 - val_accuracy: 0.9463
Epoch 6/10
26/26 [==============================] - 19s 720ms/step - loss: 0.3853 - accuracy: 0.8735 - val_loss: 0.4177 - val_accuracy: 0.8049
Epoch 7/10
26/26 [==============================] - 22s 852ms/step - loss: 0.3788 - accuracy: 0.8613 - val_loss: 0.2565 - val_accuracy: 0.9610

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 21s - loss: 0.6750 - accuracy: 0.5487 - val_loss: 0.6597 - val_accuracy: 0.5561
Epoch 2/5

Epoch 00002: saving model to weights.02-0.60.h5
26/26 - 19s - loss: 0.6238 - accuracy: 0.7092 - val_loss: 0.5974 - val_accuracy: 0.6732
Epoch 3/5

Epoch 00003: saving model to weights.03-0.61.h5
26/26 - 19s - loss: 0.5767 - accuracy: 0.7287 - val_loss: 0.6099 - val_accuracy: 0.6098
Epoch 4/5

Epoch 00004: saving model to weights.04-0.47.h5
26/26 - 19s - loss: 0.5234 - accuracy: 0.7603 - val_loss: 0.4704 - val_accuracy: 0.8390
Epoch 5/5

Epoch 00005: saving model to weights.05-0.43.h5
26/26 - 19s - loss: 0.4870 - accuracy: 0.7944 - val_loss: 0.4313 - val_accuracy: 0.7951


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 22s - loss: 0.6723 - accuracy: 0.5961 - val_loss: 0.6719 - val_accuracy: 0.5171


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 19s - loss: 0.6722 - accuracy: 0.5766 - val_loss: 0.6911 - val_accuracy: 0.4439
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 19s - loss: 0.6362 - accuracy: 0.6582 - val_loss: 0.6771 - val_accuracy: 0.4732


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 19s - loss: 0.6766 - accuracy: 0.5620 - val_loss: 0.6624 - val_accuracy: 0.5024
Epoch 2/50
26/26 - 19s - loss: 0.6297 - accuracy: 0.6606 - val_loss: 0.6009 - val_accuracy: 0.8439
Epoch 3/50
26/26 - 19s - loss: 0.5671 - accuracy: 0.7689 - val_loss: 0.5639 - val_accuracy: 0.7220
Epoch 4/50
26/26 - 19s - loss: 0.5423 - accuracy: 0.7603 - val_loss: 0.5002 - val_accuracy: 0.8195
Epoch 5/50
26/26 - 19s - loss: 0.4714 - accuracy: 0.8066 - val_loss: 0.5678 - val_accuracy: 0.6634
Epoch 6/50
26/26 - 19s - loss: 0.4125 - accuracy: 0.8321 - val_loss: 0.3448 - val_accuracy: 0.8878
Epoch 7/50
26/26 - 19s - loss: 0.3481 - accuracy: 0.8674 - val_loss: 0.2772 - val_accuracy: 0.9171
Epoch 8/50
26/26 - 19s - loss: 0.3090 - accuracy: 0.8735 - val_loss: 0.2575 - val_accuracy: 0.9171
Epoch 9/50
26/26 - 19s - loss: 0.2915 - accuracy: 0.8844 - val_loss: 0.2211 - val_accuracy: 0.9268
Epoch 10/50
26/26 - 19s - loss: 0.1910 - accuracy: 0.9538 - val_loss: 0.1461 - val_accuracy: 0.9902
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 786ms/step - loss: 0.6809 - accuracy: 0.5827 - val_loss: 0.6526 - val_accuracy: 0.5610
Epoch 2/5
26/26 [==============================] - 19s 722ms/step - loss: 0.6209 - accuracy: 0.6484 - val_loss: 0.6842 - val_accuracy: 0.5220
Epoch 3/5
26/26 [==============================] - 19s 734ms/step - loss: 0.5626 - accuracy: 0.6995 - val_loss: 0.5952 - val_accuracy: 0.6244
Epoch 4/5
26/26 [==============================] - 19s 727ms/step - loss: 0.5364 - accuracy: 0.7311 - val_loss: 0.5183 - val_accuracy: 0.7317
Epoch 5/5
26/26 [==============================] - 19s 734ms/step - loss: 0.4954 - accuracy: 0.7774 - val_loss: 0.4335 - val_accuracy: 0.8439


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.582725,0.681768,0.560976,0.652648
1,1,0.648419,0.621119,0.521951,0.684248
2,2,0.699513,0.562351,0.624390,0.595198
3,3,0.731144,0.537310,0.731707,0.518253
4,4,0.777372,0.495695,0.843902,0.433505


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 30s 1s/step - loss: 0.6803 - accuracy: 0.5766 - val_loss: 0.6907 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 19s 730ms/step - loss: 0.6635 - accuracy: 0.5401 - val_loss: 0.6708 - val_accuracy: 0.5463

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 19s 723ms/step - loss: 0.6543 - accuracy: 0.6448 - val_loss: 0.6672 - val_accuracy: 0.5463

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 19s 731ms/step - loss: 0.6493 - accuracy: 0.6375 - val_loss: 0.6645 - val_accuracy: 0.5659

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 19s 723ms/step - loss: 0.6467

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 21s 805ms/step - loss: 0.6705 - accuracy: 0.5937 - val_loss: 0.6999 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - 19s 739ms/step - loss: 0.6217 - accuracy: 0.7032 - val_loss: 0.6979 - val_accuracy: 0.4634
Epoch 3/50
26/26 [==============================] - 19s 738ms/step - loss: 0.5711 - accuracy: 0.7299 - val_loss: 0.5338 - val_accuracy: 0.8390
Epoch 4/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4920 - accuracy: 0.8075
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 19s 746ms/step - loss: 0.4888 - accuracy: 0.8102 - val_loss: 0.5486 - val_accuracy: 0.7024
Epoch 5/50
26/26 [==============================] - 19s 739ms/step - loss: 0.4381 - accuracy: 0.8394 - val_loss: 0.4260 - val_accuracy: 0.8585
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4192 - accuracy: 0.8625
Epoch 00006: ReduceLROnPlate

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 5236), started 0:15:50 ago. (Use '!kill 5236' to kill it.)